**Learning outcomes**

* Prove why we cannot design a quantum circuit that can clone arbitrary states.
* Describe the quantum teleportation protocol.

In [4]:
import numpy as np
import pennylane as qml

**Codercise I.15.1.** (a) Prepare your favourite single-qubit quantum state on the first qubit in the `state_preparation` function below. It can be anything, as long as it's a single qubit state (you can even leave the function as-is, or make it parametrized!).

Note that this quantum function will be used as a subroutine; therefore you do not need to return a measurement or create a QNode.

In [ ]:
def state_preparation(phi, theta, omega):

    # OPTIONALLY UPDATE THIS STATE PREPARATION ROUTINE

    qml.Hadamard(wires=0)
    qml.Rot(phi,theta,omega, wires = 0)

(b) Run your circuit below to see the state you will teleport. If you made a custom state preparation function, please copy it down from the cell above and use it as a replacement for the `state_preparation` function below.

In [3]:
dev = qml.device("default.qubit", wires=1)

# OPTIONALLY REPLACE THIS STATE PREPARATION ROUTINE WITH
# THE ONE FROM THE PREVIOUS EXERCISE

def state_preparation(phi, theta, omega):

    # OPTIONALLY UPDATE THIS STATE PREPARATION ROUTINE

    qml.Hadamard(wires=0)
    qml.Rot(phi,theta,omega, wires = 0)


@qml.qnode(dev)
def state_prep_only():
    state_preparation(phi = 0.1, theta=0.2, omega=0.3)
    return qml.state()

print(state_prep_only())

[0.61930934-0.13273109j 0.75978977+0.14682614j]


**Codercise I.15.2.**
Write a quantum function that prepares a shared entangled state on the second and third qubits.

Recall the teleportation circuit looks like this:
![circuit](./images/I.15.2.1.png)
This function will also be used as a subroutine, so you do not need to return any measurements.

In [5]:
def entangle_qubits():
    #ENTANGLE THE SECOND QUBIT (WIRES=1) AND THE THIRD QUBIT
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1,2])

**Codercise I.15.3.**
Implement a quantum function, `rotate_and_controls`, that performs both a change of basis, and the controlled gates at the end circuit.
![circuit](./images/I.15.2.1.png)

NOte that the `rotate_and_controls` function does not a return a measurement here; this is because when we put all the subroutines together in the full teleportation protocol, we will need that quantum function to return a measurement in order to be bound into a QNode.


In [6]:
def rotate_and_controls():
    # PERFORM THE BASIS ROTATION
    qml.CNOT(wires=[0,1])
    qml.Hadamard(wires=0)

    # PERFORM THE CONTROLLED OPERATIONS
    qml.CNOT(wires=[1,2])
    qml.CZ(wires=[0,2])

**Codercise I.15.4**
Put everything together: create a QNode that will execute your teleportation procedure, and return the quantum state!
All three of the quantum functions you wrote in the previous exercises are available here to use: `state_preparation`, `entangle_qubits`, and `rotate_and_controls`. Feel free though to use your modified version of `state_preparation`, if you prefer.

In [7]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def teleportation():

    # USE YOUR QUANTUM FUNCTIONS TO IMPLEMENT QUANTUM TELEPORTATION
    state_preparation(0.1,0.2,0.3)
    entangle_qubits()
    rotate_and_controls()
    # RETURN THE STATE
    return qml.state()


print(teleportation())

[0.30965467-0.06636554j 0.37989488+0.07341307j 0.30965467-0.06636554j
 0.37989488+0.07341307j 0.30965467-0.06636554j 0.37989488+0.07341307j
 0.30965467-0.06636554j 0.37989488+0.07341307j]


Something looks funny here - the output state we see here is a 3-qubit state. How can we determine the state of the third qubit to check whether it was correctly teleported?

**Codercise.I.15.5**
By working through the theoretical action of the circuit (see the accompanying text node for details), you can show that the combined state of the three qubits together after the procedure is
![circuit](./images/I.15.5.1.png)

With this knowledge, write a function that takes a state vector as input, and outputs the state of the third qubit as a two-element vector. Does it match the original state above?

In [8]:
def extract_qubit_state(input_state):
    """Extract the state of the third qubit from the combined state after teleportation.

    Args:
        input_state (array[complex]): A 3-qubit state of the form
            (1/2)(|00> + |01> + |10> + |11>) (a|0> + b|1>)
            obtained from the teleportation protocol.

    Returns:
        array[complex]: The state vector np.array([a, b]) of the third qubit.
    """
    # DETERMINE THE STATE OF THE THIRD QUBIT
    third_qubit = [input_state[0],input_state[1]]
    third_qubit = [2 * i for i in third_qubit]

    return third_qubit


# Here is the teleportation routine for you
dev = qml.device("default.qubit", wires=3)


# OPTIONALLY UPDATE THIS STATE PREPARATION ROUTINE
def state_preparation():
    qml.Hadamard(wires=0)
    qml.Rot(0.1, 0.2, 0.3, wires=0)


@qml.qnode(dev)
def teleportation():
    state_preparation()
    entangle_qubits()
    rotate_and_controls()
    return qml.state()

# Print the extracted state after teleportation
full_state = teleportation()
print(extract_qubit_state(full_state))

[tensor(0.61930934-0.13273109j, requires_grad=True), tensor(0.75978977+0.14682614j, requires_grad=True)]
